In [ ]:
import ee

ee.Initialize()

In [2]:
from counties import get_counties_dict

import math
from multiprocessing import Pool
import os
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import geemap.eefolium as geemap
import folium

In [4]:
def make_aoi(shape_file):
    with open (shape_file, 'r') as f:
        borders = json.load(f)

    iowa_geometry = borders['features'][0]['geometry']
    pprint(iowa_geometry)
    return ee.Geometry(iowa_geometry)

In [5]:
# Crop Season
start_date = '2019-04-01'
end_date = '2019-11-01'
area = 'iowa_simplified.geojson'



collections = [
    'NASA/ORNL/DAYMET_V4',
    'ECMWF/ERA5/DAILY',
    'NASA/GLDAS/V021/NOAH/G025/T3H',
    'USDA/NASS/CDL'
]

def get_collections(aoi, collections, start_date, end_date):
    date_filter = ee.Filter.date(start_date, end_date)
    image_collections = []
    for collection in collections:
        image_collections.append(ee.ImageCollection(collection).filterBounds(aoi))

    # inner_join = ee.Join.inner()
    return image_collections
    
aoi = ee.FeatureCollection('TIGER/2018/States').filterMetadata('NAME', 'equals', 'Iowa').geometry()
# image_collections = get_collections(aoi, collections[:2], start_date, end_date)
crop_collection = get_collections(aoi, [collections[-1]], start_date, end_date)[0].filterDate('2018-01-01', '2019-12-31')

In [6]:
crop_collection = crop_collection.map(lambda image: image.clip(aoi))
crops = crop_collection.select(['confidence']).mean()

crop_threshold = 97
ee_crop_threshold = ee.Image.constant(crop_threshold)
is_crop_mask = crops.gte(ee_crop_threshold)

cultivated = crop_collection.select(['cultivated']).mean()

cultivated_threshold = 1.5
ee_cultivated_threshold = ee.Image.constant(cultivated_threshold)
final_mask = cultivated.gte(ee_cultivated_threshold).bitwise_and(is_crop_mask)


# for i in range(len(image_collections)):
#     image_collections[i] = image_collections[i].filterDate(start_date, end_date)
#     # image_collections[i] = image_collections[i].map(lambda image: image.clip(aoi))
#     image_collections[i] = image_collections[i].map(lambda image: image.updateMask(final_mask))

In [7]:
# Crop Season
start_date = '04-01' # MM-DD
end_date = '11-01' # MM-DD
area = 'iowa_simplified.geojson'



collections = [
    'NASA/ORNL/DAYMET_V4',
    'ECMWF/ERA5/DAILY',
    'NASA/GLDAS/V021/NOAH/G025/T3H'
]
    
aoi = ee.FeatureCollection('TIGER/2018/States').filterMetadata('NAME', 'equals', 'Iowa').geometry()


def make_if_not_exists(path):
    if not os.path.exists(path):
        os.mkdir(path)

counties_dict = get_counties_dict()

cur_path = os.getcwd()
make_if_not_exists(os.path.join(cur_path, 'data'))

save_list = []

for fip_code, county_name in counties_dict.items():
    cname = county_name.replace(' ', '_')
    os.mkdir(os.path.join(cur_path, 'data', f"{cname}"))
    for year in range(2011, 2021):
        make_if_not_exists(os.path.join(cur_path, 'data', f"{cname}", f"{year}"))
        _start_date = f"{year}-{start_date}"
        _end_date = f"{year}-{end_date}"
        # date_filter = (_start_date, _end_date)
        county_aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', f'{(fip_code-19000):03}')
        for collection in collections:
            out_dir = os.path.join(cur_path, 'data', f"{cname}", f"{year}", f"{collection.replace('/', '-')}")
            make_if_not_exists(out_dir)
            clipped_ic = ee.ImageCollection(collection).filterBounds(county_aoi).filterDate(_start_date, _end_date).map(
                lambda image : image.clip(county_aoi).updateMask(final_mask))
            save_list.append((clipped_ic, out_dir, county_aoi))
            
            
        

In [8]:
num_per_thread = int(math.ceil(len(save_list)/16))

print(num_per_thread)

new_list = [save_list[i:i+num_per_thread] for i in range(0, len(save_list), num_per_thread)]

248


In [ ]:
def save_col(a):
    print(a[1])
    geemap.ee_export_image_collection(a[0], out_dir=a[1])

pool = Pool(16)

pool.map(save_col, new_list)

In [ ]:
counties_dict = get_counties_dict()


def compute_histograms(steps):
    counties_histograms = []

    for fip_code, name in counties_dict.items():
        hist_list = []
        for ic in image_collections:
            county_aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('STATEFP', 'equals', '19').filterMetadata('COUNTYFP', 'equals', f'{(fip_code-19000):03}')
            county_mins, county_maxs = (ic.min(), ic.max())
            min_reducer = ee.Reducer.min()
            max_reducer = ee.Reducer.max()
            band_mins = county_mins.reduceRegion(reducer=min_reducer, geometry=county_aoi, scale=3500)
            band_maxs = county_maxs.reduceRegion(reducer=max_reducer, geometry=county_aoi, scale=3500)

            county_mins, county_maxs = (band_mins.getInfo(), band_maxs.getInfo())

            county_ic = ic.map(lambda image : image.clip(county_aoi))
            
            band_stats = {}
            for k, v in county_mins.items():
                band_stats[k] = (v, county_maxs[k])
            
            for band_name, (band_min, band_max) in band_stats.items():
                hist_reducer = ee.Reducer.fixedHistogram(min=band_min, max=band_max, steps=steps)
                hist_list.append(county_ic.first().select([band_name]).reduceRegion(reducer=hist_reducer, geometry=county_aoi, scale=3500).getInfo())
                print(hist_list)
            break
        break
        counties_histograms.append(hist_list)
    return counties_histograms

hists = compute_histograms(1000)
        
# print(get_counties_dict())

In [ ]:
img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810')

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

In [ ]:
ic = image_collections[0]

mosaic = ic.mosaic()

mosaic_viz = {
    'min' : 0,
    'max' : 2000,
    'bands' : ['vp']
}

# (0, 86400)
# (0, 544)
# (0, 8230)

In [ ]:
map1 = geemap.Map(center=(41.7155, -93.0190), zoom=7)

map1.addLayer(image_collections[0].mosaic(), mosaic_viz, 'Daymet Mosaic')

test = crop_collection.first()

crop_viz = {
    'min' : 0,
    'max' : 100,
    'bands' : ['confidence']
}

map1.addLayer(test, crop_viz, 'Cropland')
# map1.addLayer(thresh_mask, {}, 'Mask')
map1

In [ ]:
dummy = geemap.ee_to_numpy(image_collections[0].first(), region=aoi)

In [ ]:
print(dummy)

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# # Define the visualization parameters.
# image_viz_params = {
#     'bands': ['vp'],
#     'min': 0,
#     'max': 2000,
# }

# # Define a map centered on San Francisco Bay.
# map_l8 = folium.Map(location=[41.7155, -93.0190], zoom_start=7)

# # Add the image layer to the map and display it.
# map_l8.add_ee_layer(ic.first(), image_viz_params, 'false color composite')
# display(map_l8)

In [ ]:
    #pprint(image_collections[i].first().getInfo())
    # print('!!!!!!')

    




    
# """

# # DAYMET

# 'system:index': '20190401',
# 'system:time_end': 1554163200000,
# 'system:time_start': 1554076800000

# # DAILY

# 'system:index': '20190401',
# 'system:time_end': 1554163200000,
# 'system:time_start': 1554076800000,

# # T3H
# 'system:index': 'A20190401_0000',
# 'system:time_end': 1554087600000,
# 'system:time_start': 1554076800000
# """